In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from Selection import filter_method as ft

C:\Users\Ut Luom\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Ut Luom\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Ut Luom\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Ut Luom\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
data = pd.DataFrame(np.c_[data['data'], data['target']],
                  columns= np.append(data['feature_names'], ['target']))

In [3]:
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data.target, test_size=0.2, random_state=42)

# Correlation method
loại bỏ các tính năng có độ tương quan cao với nhau

In [5]:
corr = ft.corr_feature_detect(data=X_train, threshold=0.9)
for i in corr:
    print(i, '\n')

       feature1       feature2  correlation
0  mean texture  worst texture     0.909718 

              feature1              feature2  correlation
2  mean concave points  worst concave points     0.910680
5  mean concave points        mean concavity     0.914671 

           feature1      feature2  correlation
6   perimeter error    area error     0.937271
29  perimeter error  radius error     0.973560 

      feature1         feature2  correlation
8   worst area      mean radius     0.941849
11  worst area   mean perimeter     0.943283
17  worst area        mean area     0.959723
30  worst area  worst perimeter     0.976979
32  worst area     worst radius     0.983458 



# Variance method
loại bỏ các tính năng hiển thị cùng một giá trị cho phần lớn/ tất cả các quan sát

In [6]:
quasi_constant_feature = ft.constant_feature_detect(data=X_train, threshold=0.9)

0  biến được tìm thấy gần như không đổi


In [7]:
# tạo một biến giả để thử với phương pháp trên
X_train['dummy'] = np.floor(X_train['worst smoothness']*10)
X_train.dummy.value_counts()/ np.float(len(X_train))

1.0    0.914286
0.0    0.079121
2.0    0.006593
Name: dummy, dtype: float64

In [8]:
quasi_constant_feature = ft.constant_feature_detect(data=X_train, threshold=0.9)
quasi_constant_feature

1  biến được tìm thấy gần như không đổi


['dummy']

In [9]:
# drop values
X_train.drop(labels=quasi_constant_feature, axis=1, inplace=True)

# Sắp xếp feature dựa trên AUC và MSE

In [11]:
uni_roc_auc = ft.univariate_roc_auc(X_train,y_train, X_test, y_test, threshold=0.8)
print(uni_roc_auc)

worst area                 0.917458
worst concave points       0.905830
worst radius               0.889125
worst perimeter            0.878808
area error                 0.875860
mean radius                0.865870
mean concave points        0.829021
mean concavity             0.818867
mean perimeter             0.799869
mean area                  0.797085
worst concavity            0.786276
radius error               0.765640
perimeter error            0.764330
worst compactness          0.719784
mean compactness           0.714216
worst fractal dimension    0.641828
mean texture               0.634294
concave points error       0.619227
mean symmetry              0.611202
concavity error            0.601539
worst texture              0.585653
mean smoothness            0.570259
mean fractal dimension     0.555519
worst smoothness           0.549132
compactness error          0.533082
smoothness error           0.530789
fractal dimension error    0.529152
texture error              0

In [12]:
uni_mse = ft.univariate_mse(X_train, y_train, X_test, y_test, threshold=0.4)
print(uni_mse)

symmetry error             0.486355
worst symmetry             0.449074
smoothness error           0.447368
texture error              0.440789
fractal dimension error    0.440789
compactness error          0.436404
mean fractal dimension     0.427632
mean smoothness            0.396503
worst smoothness           0.389985
worst texture              0.388158
concavity error            0.355263
mean symmetry              0.346979
mean texture               0.339912
worst fractal dimension    0.337719
concave points error       0.328947
mean compactness           0.265351
worst compactness          0.256579
radius error               0.239035
perimeter error            0.219298
worst concavity            0.197368
mean perimeter             0.186404
mean area                  0.184211
mean concavity             0.168860
mean concave points        0.162281
mean radius                0.129386
area error                 0.128899
worst perimeter            0.114035
worst radius               0